In [ ]:
# icecream==2.1.3
# huaweicloudsdkgaussdb==3.1.51
# huaweicloudsdkall==3.1.51
# esdk-obs-python==3.22.2
# python-decouple==3.8

In [ ]:
import io
import pandas as pd
from dataHuawei.utils.CloudBucket import CloudBucket
import inspect


if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

data_columns = ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount']


@data_loader
def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    server_location = 'obs.sa-brazil-1.myhuaweicloud.com'
    access_key_id ='UUPFWV82Q3J8VYVHNZBQ'
    secret_access_key = 'Ib4bDYbx5jdLrjDQulpiQRSKGPer5ffkx5ppi9ug'

    print(inspect.getfile(CloudBucket))
    with CloudBucket(access_key_id=access_key_id,
            secret_access_key=secret_access_key,
            server_location=server_location) as client:
            
            response = client.get_binary_file(bucketName='test-datalake', file_name='uber_data.csv')
            df = pd.read_csv(io.BytesIO(response.body.buffer), sep=',')
            
    return df


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'

@test
def test_data(output, *args) -> None:
    assert all([column in data_columns for column in output.columns])
    assert len(output.columns) == 19

In [1]:
import pandas as pd
from icecream import ic
from datetime import datetime
if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

# output column names
output_columns = ['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id',
       'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
       'pickup_location_id', 'dropoff_location_id', 'payment_type_id',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount']

# dataframe dictioanries
rate_code_type = {
        1:"Standard rate",
        2:"JFK",
        3:"Newark",
        4:"Nassau or Westchester",
        5:"Negotiated fare",
        6:"Group ride"
    }

payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

# Internal Tests
def test_datetime_dim(df: pd.DataFrame) -> None:
    assert not df.isnull().values.any()
    assert all([len(str(item)) == 4 and isinstance(item, int) for item in df['pick_year']])
    assert all([isinstance(date, datetime) for date in df['tpep_pickup_datetime']])
    assert all([isinstance(item, int) for item in df['pick_hour']])
    assert all([isinstance(item, int) for item in df['pick_day']])
    assert all([isinstance(item, int) for item in df['pick_month']])
    assert all([isinstance(item, int) for item in df['pick_year']])
    assert all([isinstance(item, int) for item in df['pick_weekday']])

    assert all([len(str(item)) == 4 and isinstance(item, int) for item in df['drop_year']])
    assert all([isinstance(date, datetime) for date in df['tpep_dropoff_datetime']])
    assert all([isinstance(item, int) for item in df['drop_hour']])
    assert all([isinstance(item, int) for item in df['drop_day']])
    assert all([isinstance(item, int) for item in df['drop_month']])
    assert all([isinstance(item, int) for item in df['drop_year']])
    assert all([isinstance(item, int) for item in df['drop_weekday']])

def test_passenger_count_dim(df: pd.DataFrame) -> None:
    assert not df.isnull().values.any()
    assert all([isinstance(item, int) for item in df['passenger_count']])
    assert all([isinstance(item, int) for item in df['passenger_count_id']])   


def test_trip_distance_dim(df: pd.DataFrame) -> None:
    assert not df.isnull().values.any()
    assert all([isinstance(item, int) for item in df['trip_distance_id']])
    assert all([isinstance(item, float) for item in df['trip_distance']])   

def test_rate_code_dim(rate_code_dim: pd.DataFrame) -> None:
    assert all([isinstance(id, int) for id in rate_code_dim['rate_code_id']])
    assert all([isinstance(id, int) for id in rate_code_dim['RatecodeID']])
    assert all([rate_code in rate_code_type.values() for rate_code in rate_code_dim['rate_code_name']])

def test_pickup_location_dim(pickup_location_dim: pd.DataFrame) -> None:
    assert all([isinstance(id, int) for id in pickup_location_dim['pickup_location_id']])
    assert all([isinstance(latitude, float) for latitude in pickup_location_dim['pickup_latitude']])
    assert all([isinstance(longitude, float) for longitude in pickup_location_dim['pickup_longitude']])


def test_dropoff_location_dim(dropoff_location_dim: pd.DataFrame) -> None:
    assert all([isinstance(id, int) for id in dropoff_location_dim['dropoff_location_id']])
    assert all([isinstance(latitude, float) for latitude in dropoff_location_dim['dropoff_latitude']])
    assert all([isinstance(longitude, float) for longitude in dropoff_location_dim['dropoff_longitude']])

def test_payment_type_dim(payment_type_dim: pd.DataFrame) -> None:
    assert all([isinstance(value, int) for value in payment_type_dim['payment_type']])
    assert all([isinstance(value, int) for value in payment_type_dim['payment_type_id']])
    assert any([payment_type in payment_type_name.values() for payment_type in payment_type_dim['payment_type_name']])

def test_df_datetype(output: pd.DataFrame) -> None:
    assert all([data_type == 'datetime64[ns]' for data_type in output[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].dtypes])

def test_df(output: pd.DataFrame) -> None:
    assert len(output.columns) == 17
    assert output.shape[0] > 50000
    assert all([column in output_columns for column in output.columns])

def test_trip_id(output: pd.DataFrame) -> None:
    def check_numeric_progression(id_list):
        initial_value = id_list[0]
        for idx, value in enumerate(id_list):
            if value != initial_value + idx:
                return False
        return True

    assert all(isinstance(id, int) for id in output['trip_id'])
    assert check_numeric_progression(output['trip_id'])

@transformer
def transform(df, *args, **kwargs):
    """
    Template code for a transformer block.

    Add more parameters to this function if this block has multiple parent blocks.
    There should be one parameter for each output variable from each parent block.

    Args:
        data: The output from the upstream parent block
        args: The output from any additional upstream blocks (if applicable)

    Returns:
        Anything (e.g. data frame, dictionary, array, int, str, etc.)
    """
    # Specify your transformation logic here
    # df_datetype & test
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    test_df_datetype(df)
    # trip_id
    df = df.drop_duplicates().reset_index(drop=True)
    df['trip_id'] = df.index

    # datetime_dim schema & test
    datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
    datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
    datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
    datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
    datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
    datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
    datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

    datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
    datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
    datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
    datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
    datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
    datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

    datetime_dim['datetime_id'] = datetime_dim.index
    datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

    test_datetime_dim(datetime_dim)

    # passenger_count_dim schema & test
    passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
    passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
    passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]

    test_passenger_count_dim(passenger_count_dim)

    # trip_distance_dim schema & test
    trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
    trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
    trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']] 

    test_trip_distance_dim(trip_distance_dim)

    # rate_code_dim schema & test
    rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
    rate_code_dim['rate_code_id'] = rate_code_dim.index
    rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
    rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

    test_rate_code_dim(rate_code_dim)

    # pickup_location_dim schema & test
    pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)
    pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
    pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 

    test_pickup_location_dim(pickup_location_dim)

    # dropoff_location_dim schema & test
    dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)
    dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
    dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]

    test_dropoff_location_dim(dropoff_location_dim)


    # payment_type_dim schema & test
    payment_type_dim = df[['payment_type']].reset_index(drop=True)
    payment_type_dim['payment_type_id'] = payment_type_dim.index
    payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
    payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]

    test_payment_type_dim(payment_type_dim)

    # fact_table schema & test
    fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

    test_df(fact_table)
    test_trip_id(fact_table)

    return fact_table
    # return "success"
    # return {
    #     "datetime_dim": datetime_dim.to_dict(orient="dict"),
    #     "passenger_count_dim": passenger_count_dim.to_dict(orient="dict"),
    #     "trip_distance_dim": trip_distance_dim.to_dict(orient="dict"),
    #     "rate_code_dim": rate_code_dim.to_dict(orient="dict"),
    #     "pickup_location_dim": pickup_location_dim.to_dict(orient="dict"),
    #     "dropoff_location_dim": dropoff_location_dim.to_dict(orient="dict"),
    #     "payment_type_dim": payment_type_dim.to_dict(orient="dict"),
    #     "fact_table": fact_table.to_dict(orient="dict")
    # }


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'



# 41941866

ModuleNotFoundError: No module named 'mage_ai'